# Frequentist vs Bayesian significance

In [33]:
!pip install iminuit

For a simple counting experiment, the expected background event is $b$ and the observed event is $n$.
The best estimator for signal event $s$ is:
$$s=n-b.$$ In this exercise, we will implement Frequentist significance calculation.

$$ p-value =  \int_{q_{0,n}}^{\infty} f(q_0|b) dq_0 $$
$$Z_{0, Frequentist} = \mathrm{Z score\ 1-tailed\ (p-value)} $$
, where $f(q_0|b)$ is distribution of test statistics $q_0$ in background only hypothesis.
The test statistics is defined as likelihood ratio between backogrund only model and the best fit model
$$ q_0 = -2 \mathrm{ln} \frac{L(n|s=0,b)}{L(n|s,b)} $$
In the simple counting experiment, the Likelihood $L$ is Poisson distribution. The test statistics can be written as
$$ q_0 = -2 \mathrm{ln} \frac{\mathrm{Poisson}(n|b)}{\mathrm{Poisson}(n|s+b)}$$




In [34]:
import numpy as np
from scipy.stats import poisson, norm

# Define test statistics q_0 for Frequentist approach
# 1. We require signal event s > 0 for positive signal yield.
#    Therefore, the test statistics q_0 is 0 if N_obs <= Nb
# 2. Compute two Poisson loglikelihood of
#    a) background only model
#    b) signal+background model
#    Evaluate -2 log likelihood ratio between a) and b)
def q0(N_obs, Nb):
    # Write your code here
    # s_hat = max(0, N_obs - Nb); 当 N_obs <= Nb 时 q0 = 0
    if N_obs <= Nb:
        q0_out = 0.0
    else:
        n = float(N_obs)
        b = float(Nb)
        # -2 ln λ = 2 [ n ln(n/b) - (n - b) ]
        q0_out = 2.0 * (n * np.log(n / b) - (n - b))
    return q0_out


In [35]:
def FreqnetistZ0(N_obs, N_b, ntoys=200000, rng=None):
    """
    Estimate p-value and Z under background-only hypothesis (H0)
    using toy Monte Carlo experiments.
    """
    rng = np.random.default_rng(rng)
    # Step1: Generate toy experiments under H0
    ns = rng.poisson(lam=N_b, size=ntoys)
    q0_b_dist = np.array([q0(int(n_i), N_b) for n_i in ns])

    # Step2: Compute observed q0
    q0_obs = q0(int(N_obs), N_b)

    # Step3: Compute p-value
    p_value = float(np.mean(q0_b_dist >= q0_obs - 1e-15))

    # Step4: Convert to Z
    Z = norm.isf(p_value) if p_value > 0 else float('inf')

    return Z, p_value, q0_obs, q0_b_dist


Now, let's apply our code for numerical calculations.

Consider the case that backogrund only model with yields b=0.5 and observed events n=5.

Calclate discovery significance.

In [36]:
Nobs = 5
Nb = 0.5

q0_obs = q0(Nobs, Nb)
Z_asymp = np.sqrt(q0_obs)

Z_toy, pval, q0_obs2, q_b = FreqnetistZ0(Nobs, Nb)

print("q0_obs =", q0_obs)
print("Z_asymp =", Z_asymp)
print("Z_toy   =", Z_toy)
print("p-value =", pval)


q0_obs = 14.02585092994046
Z_asymp = 3.7451102693966782
Z_toy   = 3.6331335509472495
p-value = 0.00014


Q1: How is the number compared to the Baysian signficance from homework5?
Ans (write here):
The Frequentist Z from this homework is very close to the Bayesian significance from Homework 5.
For the case n = 5, b = 0.5, both give Z ≈ 3.7, showing strong evidence for signal.
For n = 5, b = 4, both give Z ≈ 0.3–0.4, indicating no significant excess.
Overall, the two approaches are consistent within small numerical differences due to different interpretations of uncertainty.

Consdier a background only model with yields b=4 and observed events n=5. Calculate Baysian signifiance and Frequqnt signficance, and compare the results.

In [37]:
Nobs = 5
Nb = 4.0

q0_obs = q0(Nobs, Nb)
Z_asymp = np.sqrt(q0_obs)

Z_toy, pval, q0_obs2, q_b = FreqnetistZ0(Nobs, Nb)

print("q0_obs =", q0_obs)
print("Z_asymp =", Z_asymp)
print("Z_toy   =", Z_toy)
print("p-value =", pval)


q0_obs = 0.23143551314209754
Z_asymp = 0.48107745025317655
Z_toy   = 0.3230465349543169
p-value = 0.37333


In [38]:


def BayesianZ0(N_obs, N_b, pi0=0.5, tau=None, rtol=1e-10):
    """
    Bayesian 'significance' against H0:s=0 for Poisson(n | s+b),
    using a spike-and-slab prior:
        P(H0) = pi0,  s = 0
        P(H1) = 1 - pi0,  s ~ Exp(mean=tau)
    Returns:
        Z_bayes, post_H0, BF10
    """
    n = int(N_obs)
    b = float(N_b)
    if tau is None:
        tau = max(1.0, b)

    # Likelihood under H0
    L0 = poisson.pmf(n, b)

    # Marginal likelihood under H1
    def integrand(s):
        lam = s + b
        if lam <= 0.0:
            return 0.0
        return poisson.pmf(n, lam) * (1.0 / tau) * np.exp(-s / tau)

    m1, _ = quad(integrand, 0.0, np.inf, epsrel=rtol, limit=200)

    # Bayes factor
    if L0 == 0.0:
        BF10 = np.inf if m1 > 0 else 0.0
    else:
        BF10 = m1 / L0

    denom = pi0 * L0 + (1.0 - pi0) * m1
    post_H0 = (pi0 * L0 / denom) if denom > 0 else 1.0

    Z_bayes = norm.isf(post_H0) if 0.0 < post_H0 < 1.0 else (np.inf if post_H0 <= 0 else 0.0)

    return Z_bayes, post_H0, BF10


In [39]:
#Test
Z_bayes, post_H0, BF10 = BayesianZ0(5, 4, pi0=0.5, tau=4)
print("Bayesian Z =", Z_bayes, "P(H0|n) =", post_H0, "BF10 =", BF10)

Bayesian Z = -0.2211609688737606 P(H0|n) = 0.5875164504606137 BF10 = 0.7020799999999976


In [40]:
import numpy as np

q0_value = q0(5, 4)
sqrt_q0 = np.sqrt(q0_value)
print(f"The square root of q0 for N_obs=5 and N_b=4 is: {sqrt_q0:.2f}")

The square root of q0 for N_obs=5 and N_b=4 is: 0.48


Compare and explain the core difference between Frequentist and Bayesian significant.
In essence:

Frequentist significance quantifies how unlikely your data is under the null hypothesis.
Bayesian significance quantifies how likely a hypothesis (like the presence of a signal) is given your data and prior beliefs.

Frequentist significance measures the probability of observing data as extreme as (or more extreme than) what was measured, assuming the null hypothesis is true. It focuses on the behavior of possible data under H_0.

Bayesian significance, on the other hand, evaluates the probability of the hypothesis itself—such as the presence of a signal—given the observed data and prior assumptions. It combines prior beliefs with the evidence provided by the data to update the credibility of each hypothesis.

In short, the Frequentist approach tests data under a fixed hypothesis, while the Bayesian approach tests hypotheses given observed data.